In [ ]:
# standard imports
import numpy as np
import math
import scipy
import os
import csv
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt

In [ ]:
# custom class imports
from node import node
from cell import cell
from mesh import exp_mesh
from derivatives import derivatives
from fluid_problem import isentropic_fluid_problem

# utility functions
from utilities import *

# Setting Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./lam_cyl_comp_air/mesh_cylinder_quad.su2",file_format="su2")
# define mid point for rotational symmetric mesh
mid_point = np.array([0.5,0]) 
# convert mesh into custom class exp_mesh
c_mesh = exp_mesh(meshio_mesh,mid_point)
# declare problem instance
comp_air = isentropic_fluid_problem(c_mesh)

# Loading Filedata

In [ ]:
# reading file data
data_dir="./lam_cyl_comp_air/restart/"
data_dict = get_flow_data(data_dir,1000,1,1102)

# Node based POD Decomposition

In [ ]:
# configure state data 
q = np.vstack([data_dict["u"],data_dict["v"],data_dict["a"]])
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),q.shape[1],axis=1)

U, S, Vh = np.linalg.svd(q_fluc, full_matrices=False)

# reconstruction threshold
epsilon = 0.97 # [0,1]
acc = 0
num = 0
while acc < epsilon:
    num+=1
    acc = sum(S[:num])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(num))
if num % 2 ==1:
    num+=1

coeffs = np.matmul(np.transpose(U[:,:num]),q_fluc)

In [ ]:
plt.plot(S)
plt.grid()
plt.yscale('log')

In [ ]:
x,y = get_x_y(data_dir)
plot_eigenflows(x,y,U,c_mesh.n,num)

In [ ]:
# set timestep 
dt = 100
t = np.linspace(0,(U.shape[1]-1)*dt,U.shape[1])

# Galerkin model based on discrete reconstruction as q = \bar{q}+\sum_i^n a_i(t)*\phi_i(x)
fig,ax = plt.subplots(3,2,figsize=(15,10))
fig.tight_layout(pad=3.0)

# add plots over full time domain
ax[0][0].plot(t,coeffs[0,:])
ax[0][1].plot(t,coeffs[1,:])
ax[1][0].plot(t,coeffs[2,:])
ax[1][1].plot(t,coeffs[3,:])
ax[2][0].plot(t,coeffs[4,:])
ax[2][1].plot(t,coeffs[5,:])

# set fixed time window to plot
ax[0][0].set_xlim([0,t[-1]])
ax[0][1].set_xlim([0,t[-1]])
ax[1][0].set_xlim([0,t[-1]])
ax[1][1].set_xlim([0,t[-1]])
ax[2][0].set_xlim([0,t[-1]])
ax[2][1].set_xlim([0,t[-1]])

# Set titles
ax[0][0].title.set_text("Activation for Eigenflow 0")
ax[0][1].title.set_text("Activation for Eigenflow 1")
ax[1][0].title.set_text("Activation for Eigenflow 2")
ax[1][1].title.set_text("Activation for Eigenflow 3")
ax[2][0].title.set_text("Activation for Eigenflow 4")
ax[2][1].title.set_text("Activation for Eigenflow 5")

# Reconstruction of data

In [ ]:
# reconstruction 
rec = np.zeros(q.shape)
rec += np.repeat(np.expand_dims(q_avg,1),coeffs.shape[1],1)
rec +=  np.matmul(U[:,:num],coeffs[:num,:])

In [ ]:
n = c_mesh.n
sample = 0
# computing total velocity!
data = np.sqrt(np.multiply(rec[:n,sample],rec[:n,sample])+np.multiply(rec[n:2*n,sample],rec[n:2*n,sample]))
real_data = np.sqrt(np.multiply(q[:n,sample],q[:n,sample])+np.multiply(q[n:2*n,sample],q[n:2*n,sample]))
# visualizing
fig,ax = plt.subplots(1,2,figsize=(20,6))
plot_cylinder(x,y,data,cmap=get_cmap(False),ax=ax[0],clip=False)
plot_cylinder(x,y,real_data,cmap=get_cmap(False),ax=ax[1],clip=False)

## Galerkin System

In [ ]:
# compute cell based eigenvectors
U_cell = np.empty((3*c_mesh.N,num))

for i in range(num):
   u = c_mesh.compute_cell_values_from_node_data(U[:c_mesh.n,i])
   v = c_mesh.compute_cell_values_from_node_data(U[c_mesh.n:2*c_mesh.n,i])
   a = c_mesh.compute_cell_values_from_node_data(U[2*c_mesh.n:3*c_mesh.n,i])
   U_cell[:,i] = np.concatenate([u,v,a])


In [ ]:
# Computation of Galerkin system parameters
Qavg = comp_air.Q_operator(q_avg,q_avg)
Lavg = comp_air.L_operator(q_avg)

# initialie arrays for parameters
b1 = np.empty(num)
b2 = np.empty(num)
L1 = np.empty((num,num))
L2 = np.empty((num,num))
Q = [np.empty((num,num,)) for x in range(num)]

# defining parameter files
coef_dir = "./coefficients/"
b1_filename = coef_dir+"b1"
b2_filename = coef_dir+"b2"
L1_filename = coef_dir+"L1"
L2_filename = coef_dir+"L2"
Q_filename = coef_dir+"Q"

# set computation flag
compute_coefficients = True

if os.path.exists(b1_filename):
    b1 = np.load(b1_filename)
if os.path.exists(b2_filename):
    b2 = np.load(b2_filename)
if os.path.exists(L1_filename):
    L1 = np.load(L1_filename)
if os.path.exists(L2_filename):
    L2 = np.load(L2_filename)
if os.path.exists(Q_filename):
    Q = np.load(Q_filename) 

if compute_coefficients:
    # compute L and Q operators for projection
    Q_tmp1 = np.empty((num,3*len(c_mesh.cells)))
    Q_tmp2 = np.empty((num,3*len(c_mesh.cells)))
    Q_tmp3 = np.empty((num,num,3*len(c_mesh.cells)))
    L_tmp = np.empty((num,3*len(c_mesh.cells)))
    for i in tqdm(range(num)):
        Q_tmp1[i] = comp_air.Q_operator(q_avg,U[:,i])
        Q_tmp2[i] = comp_air.Q_operator(U[:,i],q_avg)
        L_tmp[i] = comp_air.L_operator(U[:,i])
        for j in range(num):
            Q_tmp3[i,j] = comp_air.Q_operator(U[:,i],U[:,j])
    
    # compute ODE coefficients
    for k in tqdm(range(num)):
        b1[k] = comp_air.compute_inner_product(Lavg,U_cell[:,k])
        b2[k] = comp_air.compute_inner_product(Qavg,U_cell[:,k])
        for i in range(num):
            L1[k,i] = comp_air.compute_inner_product(L_tmp[i],U_cell[:,k])
            L2[k,i] = comp_air.compute_inner_product(Q_tmp1[i]+Q_tmp2[i],U_cell[:,k])
            for j in range(num):
                Q[k][i,j] = comp_air.compute_inner_product(Q_tmp3[i,j],U_cell[:,k])
        
    # saving coefficient arrays to files
    np.save(b1_filename,b1)
    np.save(b2_filename,b2)
    np.save(L1_filename,L1)
    np.save(L2_filename,L2)
    np.save(Q_filename,Q)

# Solving ODE system

In [ ]:
# initial conditions for the activations are obtained as activations on the initial timeframe of the data matrix
a0 = coeffs[:,0] #initial conditions
dt = 1
t0 = 0
tmax = 4000
sampling_span = np.linspace(t0,tmax-dt,num=int(tmax/dt))
# static viscosity
nu = 1.516e-5

# defining ode function
def galerkin_system(t,a):
    # set global variables references
    global Q, L1, L2, b1, b2, nu
    a_dot = np.empty_like(a)
    for k in range(a_dot.shape[0]):
        # the vector notation makes np.matmul automatically reduce to a one dimensional output therefore normal multiplication can be used to get vector shape
        # the indexed vectors L1 and L2 form columnvectors (automatically)
        a_dot[k] = nu * b1[k] + b2[k] + np.matmul((nu*L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q[k]),np.expand_dims(a,1))
    return a_dot.flatten()

In [ ]:
from scipy.integrate import solve_ivp
sol = solve_ivp(galerkin_system,(t0,tmax),a0,method='RK45',t_eval=sampling_span,atol=1e-12,rtol=1e-12)
sol

In [ ]:
def reconstruct_flow(activations,U,avg,num,rang=(0,100)):
    low,high = rang
    flow = np.repeat(np.expand_dims(avg,1),high-low,1)
    for i in range(num):
         flow += np.outer(U[:,i],activations[i,low:high])
    return flow

In [ ]:
# Galerkin model based on discrete reconstruction as q = \bar{q}+\sum_i^n a_i(t)*\phi_i(x)
fig,ax = plt.subplots(3,2,figsize=(15,10))
fig.tight_layout(pad=3.0)

t = sampling_span

# add plots over full time domain
ax[0][0].plot(t,sol.y[0,:len(t)])
ax[0][1].plot(t,sol.y[1,:len(t)])
ax[1][0].plot(t,sol.y[2,:len(t)])
ax[1][1].plot(t,sol.y[3,:len(t)])
ax[2][0].plot(t,sol.y[4,:len(t)])
ax[2][1].plot(t,sol.y[5,:len(t)])

# Set titles
ax[0][0].title.set_text("Activation for Eigenflow 0")
ax[0][1].title.set_text("Activation for Eigenflow 1")
ax[1][0].title.set_text("Activation for Eigenflow 2")
ax[1][1].title.set_text("Activation for Eigenflow 3")
ax[2][0].title.set_text("Activation for Eigenflow 4")
ax[2][1].title.set_text("Activation for Eigenflow 5")

In [ ]:
rec = reconstruct_flow(sol.y,U,q_avg,num,rang=(2000,2300))

In [ ]:
sample = 100
# computing total velocity!
data = np.sqrt(np.multiply(rec[:n,sample],rec[:n,sample])+np.multiply(rec[n:2*n,sample],rec[n:2*n,sample]))
real_data = np.sqrt(np.multiply(q[:n,sample],q[:n,sample])+np.multiply(q[n:2*n,sample],q[n:2*n,sample]))
# visualizing
fig,ax = plt.subplots(1,2,figsize=(20,6))
plot_cylinder(x,y,data,cmap=get_cmap(False),ax=ax[0],clip=False)
plot_cylinder(x,y,real_data,cmap=get_cmap(False),ax=ax[1],clip=False)